# Tutorial: Advective-Diffusive Heat Transport in Frcatured Rock (Lauwerier 1955)

## Problem description

In [1]:
from IPython.display import Image, display, HTML

# Create the HTML structure for side-by-side layout
side_by_side = """
<div style="display: flex;">
    <div style="flex: 50%; padding: 10px;">
        <img src="./lauwerier-problem.png" style="width:100%; border: 1px solid #ddd; border-radius: 5px;">
    </div>
    <div style="flex: 50%; padding: 10px; font-family: Arial;">
        <h3>Governing processes of the Lauwerier problem</h3>
        <p>The figure on the left shows a sketch of the model geometry and heat transport processes:</p>
        <ul>
            <li>heat advection in the rock fracture</li>
            <li>heat conduction in the rock matrix</li>
        </ul>
    </div>
</div>
"""

# Display the constructed layout
display(HTML(side_by_side))

## Governing equations

Heat transport equations of the fracture-matrix system:

\begin{eqnarray}
\frac{b}{2} \rho_w c_w \frac{\partial T_f}{\partial t} 
+ 
\frac{b}{2} \rho_w c_w v_w\frac{\partial T_f}{\partial x}
- 
\kappa_m \left( \frac{\partial T_m}{\partial y} \right)_{y=b_f/2}
&=&
0
\\
\kappa_m \frac{\partial^2 T_m}{\partial y^2} &=& \left( (1-\phi) \rho_r c_r + \phi \rho_w c_w \right) \frac{\partial T_m}{\partial t}
\end{eqnarray}

Analytical solution by Laplace transformation:

\begin{eqnarray}
T_m &=& T_1 + (T_0-T_1) \, \text{erfc} \left( \frac{\xi+\eta-1}{2\sqrt{\theta(\tau-\xi)}}\right) U(\tau-\xi)
\\
T_f &=& T_1 + (T_0-T_1) \, \text{erfc} \left( \frac{\xi}{2\sqrt{\theta(\tau-\xi)}}\right) U(\tau-\xi)
\end{eqnarray}

with

\begin{equation}
\xi = \frac{x \kappa_m}{(b_f/2)^2 \rho_w c_w v_w}
\quad , \quad
\eta = \frac{y}{b_f/2}
\quad , \quad
\tau = \frac{t \kappa_m}{(b_f/2)^2 \rho_W c_w}
\quad , \quad
\theta \frac{\rho_w c_w}{(1-\phi) \rho_r c_r + \phi \rho_w c_w}
\end{equation}

---
## Material and systems conditions
---
Workflow steps:
- [ ] Material properties
- [ ] System conditions

### Material properties

In [2]:
rho_eff = 1000
cp_eff = 2000
lambda_eff = 2.2
phi = 0.15  # Porosity
alpha = lambda_eff / (rho_eff * cp_eff)  # Thermal diffusivity

### System conditions

In [3]:
T_i = 300  # Initial temperature
T_0 = 330  # Boundary condition left
v_x = 1.5e-6  # Groundwater velocity
delta_t = 0.5 * 86400
max_time = 500 * 86400

---
## Numerical solution
---
Workflow steps:
- Pre-processing
    - [x] Meshing step: `GMSH`
    - [ ] Model set-up: `PRJ` - OGS project file
- Simulation
- Post-processing

### Meshing step

In [ ]:
import gmsh
import sys

gmsh.initialize(sys.argv)
gmsh.model.add("my_model")
# ... Your geometry creation code here (add points, lines, surfaces) ...
# Example: A simple rectangle
lc1 = 1.0
lc2 = 2.5
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1)
p2 = gmsh.model.geo.addPoint(100, 0, 0, lc1)
p3 = gmsh.model.geo.addPoint(100, 25, 0, lc2)
p4 = gmsh.model.geo.addPoint(0, 25, 0, lc2)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)
curve_loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
surface = gmsh.model.geo.addPlaneSurface([curve_loop])

domain_marker = 1
surfaces = gmsh.model.getEntities(dim=2)
gmsh.model.addPhysicalGroup(2, [curve_loop], domain_marker)
gmsh.model.setPhysicalName(2, domain_marker, "MyDomain")

# Set recombination option BEFORE generating the mesh
##gmsh.option.setNumber("Mesh.RecombineAll", 1) #quad mesh
# Synchronize before meshing
gmsh.model.geo.synchronize()
# Generate the 2D mesh
gmsh.model.mesh.generate(2)  # Argument '2' for 2D mesh
# Write the mesh to disk
gmsh.write("output_mesh.msh")
gmsh.write("output_mesh.vtk")
# Launch GUI to view (optional)
gmsh.fltk.run()
gmsh.finalize()

In [5]:
import pyvista as pv
# Load your VTK file
mesh = pv.read('output_mesh.vtk')
# Get mesh information
print(f"Number of points: {mesh.n_points}")
print(f"Number of cells: {mesh.n_cells}")
#print(f"Cell types: {mesh.cell_types}")
# See the names of all data arrays in your file
print("Available point data arrays:", mesh.point_data.keys())
print("Available cell data arrays:", mesh.cell_data.keys())
# Plot as wireframe
mesh.plot(show_edges=True, color='white', line_width=1)
mesh.save('output_mesh.vtu')

Number of points: 1314
Number of cells: 2454
Available point data arrays: []
Available cell data arrays: ['CellEntityIds']


Widget(value='<iframe src="http://localhost:55855/index.html?ui=P_0x26daca76120_0&reconnect=auto" class="pyvis…

 JS Error => error: Uncaught SyntaxError: Unexpected string
 JS Error => error: Uncaught SyntaxError: Unexpected string


In [6]:
import pyvista as pv
import numpy as np

# 1. Load your mesh
mesh = pv.read('output_mesh.vtk')

# 2. Extract ALL edges from the mesh
# Note: For large meshes, this can be memory-intensive.
edges = mesh.extract_all_edges()

# 3. Get the points that define each edge segment
# edges.lines connects points in pairs: [2, point1_id, point2_id, 2, point3_id, point4_id, ...]
lines = edges.lines

# 4. Calculate the length of every edge segment
edge_lengths = []
for i in range(0, lines.size, 3):
    # The structure is [num_points, id1, id2]
    pt1_id, pt2_id = lines[i+1], lines[i+2]
    pt1 = edges.points[pt1_id]
    pt2 = edges.points[pt2_id]
    # Compute Euclidean distance
    length = np.linalg.norm(pt2 - pt1)
    edge_lengths.append(length)

# 5. Find the smallest length
if edge_lengths:
    min_length = min(edge_lengths)
    print(f"Smallest edge length: {min_length:.6e}")
    print(f"Number of edges analyzed: {len(edge_lengths)}")
else:
    print("No edges were extracted.")

Smallest edge length: 6.842518e-01
Number of edges analyzed: 3767


In [7]:
import pyvista as pv
mesh = pv.read('output_mesh.vtk')
plotter = pv.Plotter()
# Option 1: Show mesh edges on a solid surface
plotter.add_mesh(mesh, show_edges=True, edge_color='black', 
                 line_width=2, color='lightgray', opacity=1.0)
# Option 2: Pure wireframe (no faces)
plotter.add_mesh(mesh, style='wireframe', color='black', line_width=2)
# Option 3: Show points/nodes
plotter.add_mesh(mesh, style='points', color='red', point_size=5)
plotter.show()

Widget(value='<iframe src="http://localhost:55855/index.html?ui=P_0x26db56239d0_1&reconnect=auto" class="pyvis…

For this discretisation, the numerical solution approximates well to the analytical one. Finer resolutions in the time discretisation reduce the deviations considerably. In this benchmark it is easy to see that too coarse resolutions (especially in the time discretisation) yield very plausible results, which can, however, deviate considerably from the exact solution. An analysis of the von Neumann stability criterion is worthwhile here. This criterium demands

$$\text{Ne}=\frac{\alpha\Delta t}{\left(\Delta x\right)^2}\leq\frac{1}{2}$$

Evaluated for the problem at hand, the following value results:

The elements located at approximately $x>1\text{m}$ satisfy this criterion, therefore the solution presented here can be accepted as an approximation of the exact solution.